In [1]:
import pickle
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from modules import tokenize_and_lemmatize

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package wordnet to /Users/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load data

In [2]:
vacancy_name = 'Аналитик данных/data analyst'
vacancy_area = 'Москва'
vacancy_experience = 'От 1 года до 3 лет'
vacancy_employment = 'Полная занятость'
vacancy_schedule = 'Полный день'
vacancy_description = 'Твои задачи:• Формирование баз данных для КЦ (сбор, консолидация, проверка качества и актуальности).• Работа с источниками данных (поиск, анализ конверсии, отчётность) через собственную платформу.• Аналитика и отчётность (отчёты, выводы, рекомендации).• Поддержка контента и скриптов (анализ и корректировки).Что мы ждем:Hard skills:• Понимание метрик аналитики и маркетинга (CTR, CR, CPL, ROI, UTM).• • Навыки сбора, очистки и визуализации данных.. Продвинутое владение Excel / Google Таблицами (формулы, сводные таблицы, Power Query).· Опыт работы с системами визуализации данных (например, Power BI или Google Looker Studio) будет преимуществом.· Понимание ключевых маркетинговых метрик: конверсия (CR), стоимость лида (CPL), возврат на инвестиции (ROI).· Навыки работы с большими массивами информации: сбор, очистка, структурирование.Личностные Soft Skills:· Внимательность к деталям и аккуратность.· Системный и аналитический подход к решению задач.· Ответственность и нацеленность на конкретный результат.· Инициативность.Условия:Зарплата: от 80 000 ₽ (обсуждается по итогам собеседования).(Выплаты 3 раза в месяц!)· Тестовый период: 3 дня (полностью оплачиваются).· Возможности для роста: Карьерный рост до ведущего аналитика или менеджера по маркетинговым каналам.KNAM.PRO — это про результат, рост и большие деньги.Если хочешь быть в команде, которая делает рынок лидогенерации №1 в России — жми “Откликнуться”!'

In [3]:
input_dict = {
    'vacancy_name': vacancy_name,
    'vacancy_area': vacancy_area,
    'vacancy_experience': vacancy_experience,
    'vacancy_employment': vacancy_employment,
    'vacancy_schedule': vacancy_schedule,
    'vacancy_description': vacancy_description
    }

In [4]:
df_resumes = pd.read_csv('df_resumes.csv')

In [5]:
df_vacancy = pd.DataFrame([input_dict])

In [6]:
df = df_vacancy.merge(df_resumes, how='cross')

In [7]:
df_resumes[df_resumes['resume_id'] == 116651504]

,resume_id,resume_title,resume_specialization,resume_last_position,resume_last_experience_description,resume_last_company_experience_period,resume_skills,resume_education,resume_courses,resume_salary,resume_age,resume_total_experience,resume_experience_months,resume_location,resume_gender,resume_applicant_status,resume_last_company_experience_months
653,116651504,Data scientist/ML Engineer,"['Аналитик', 'Дата-сайентист', 'Программист, р...",Data Scientist,"- Полный цикл процесса сбора, очистки и подгот...",10 месяцев,"[' Python', 'Machine Learning', 'Data Science'...",['ГУ «РНПЦ трансфузиологии и медицинских биоте...,"['TeachMeSkills, Python разработчик']",0.0,34.0,8 лет 9 месяцев,105.0,Минск,Мужчина,Рассматривает предложения,10


In [8]:
df[df['resume_id'] == 116651504]

,vacancy_name,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,vacancy_description,resume_id,resume_title,resume_specialization,resume_last_position,...,resume_education,resume_courses,resume_salary,resume_age,resume_total_experience,resume_experience_months,resume_location,resume_gender,resume_applicant_status,resume_last_company_experience_months
653,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,116651504,Data scientist/ML Engineer,"['Аналитик', 'Дата-сайентист', 'Программист, р...",Data Scientist,...,['ГУ «РНПЦ трансфузиологии и медицинских биоте...,"['TeachMeSkills, Python разработчик']",0.0,34.0,8 лет 9 месяцев,105.0,Минск,Мужчина,Рассматривает предложения,10


# Cross features

In [9]:
df['location_matching'] = df.apply(lambda row: 1 if row['vacancy_area'] == row['resume_location'] else 0, axis=1)

In [10]:
def resume_skill_count_in_vacancy(row):
    count = 0
    skill_list = row['resume_skills'].replace('[', '').replace(']', '').replace("'", "").split(', ')
    for i in skill_list:
        if i in row['vacancy_description']:
            count += 1
    return count

df['resume_skill_count_in_vacancy'] = df.apply(resume_skill_count_in_vacancy, axis=1)

In [11]:
def last_position_in_vacancy(row):
    bow = []
    seps = [' ', '-', '_']
    for sep in seps:
        bow += row['resume_last_position'].split(sep=sep)
        bow = list(set(bow))
    
    c = 0
    for word in bow:
        if word in row['vacancy_description']:
            c +=1
    
    return c / len(bow)

df['last_position_in_vacancy'] = df.apply(last_position_in_vacancy, axis=1)

In [12]:
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

In [13]:
vacancy_description = df['vacancy_description'].unique().tolist()
vacancy_embedding = tfidf_vectorizer.transform(vacancy_description)

In [14]:
experience_embeddings = sp.load_npz('experience_embeddings.npz')

In [15]:
def calculate_cosine_similarity(vacancy_embedding, experience_embeddings):
    """Вычисление косинусного сходства"""
    similarities = []
    
    for i in range(experience_embeddings.shape[0]):
        exp_row = experience_embeddings[i]
        
        similarity = cosine_similarity(vacancy_embedding[0], exp_row)[0][0]
        similarities.append(similarity)
    
    return similarities

In [16]:
similarity_scores = calculate_cosine_similarity(vacancy_embedding, experience_embeddings)
    
df['similarity_score_tfidf'] = similarity_scores

In [17]:
features = [
    'vacancy_area',
    'vacancy_experience',
    'vacancy_employment', 
    'vacancy_schedule',
    'resume_salary',
    'resume_age', 
    'resume_experience_months',
    'resume_location',
    'resume_gender', 
    'resume_applicant_status', 
    'resume_last_company_experience_months', 
    'location_matching',
    'resume_skill_count_in_vacancy',
    'last_position_in_vacancy',
    'similarity_score_tfidf'
]
df[features]

,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,resume_salary,resume_age,resume_experience_months,resume_location,resume_gender,resume_applicant_status,resume_last_company_experience_months,location_matching,resume_skill_count_in_vacancy,last_position_in_vacancy,similarity_score_tfidf
0,Москва,От 1 года до 3 лет,Полная занятость,Полный день,0.0,65.0,228.0,Москва,Мужчина,Рассматривает предложения,76,1,0,0.00,0.064805
1,Москва,От 1 года до 3 лет,Полная занятость,Полный день,0.0,43.0,208.0,Москва,Мужчина,Рассматривает предложения,8,1,0,0.00,0.007989
2,Москва,От 1 года до 3 лет,Полная занятость,Полный день,200000.0,52.0,360.0,Москва,Женщина,NDT,136,1,0,0.00,0.025913
3,Москва,От 1 года до 3 лет,Полная занятость,Полный день,500000.0,56.0,356.0,Красноярск,Мужчина,Рассматривает предложения,135,0,0,0.00,0.087667
4,Москва,От 1 года до 3 лет,Полная занятость,Полный день,0.0,48.0,301.0,Moscow,Мужчина,NDT,0,0,0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20713,Москва,От 1 года до 3 лет,Полная занятость,Полный день,200000.0,50.0,308.0,Moscow,Мужчина,NDT,0,0,0,0.00,0.000000
20714,Москва,От 1 года до 3 лет,Полная занятость,Полный день,80000.0,40.0,103.0,Москва,Мужчина,NDT,47,1,0,0.00,0.011738
20715,Москва,От 1 года до 3 лет,Полная занятость,Полный день,40000.0,41.0,134.0,Москва,Мужчина,NDT,2,1,0,0.25,0.000000
20716,Москва,От 1 года до 3 лет,Полная занятость,Полный день,0.0,58.0,220.0,Москва,Женщина,NDT,91,1,0,0.00,0.036427


In [18]:
with open('feature_engineering_with_best_optuna_lr.pkl', 'rb') as f:
    model = pickle.load(f)

In [19]:
y_pred = model.predict(df[features])
y_pred_proba = model.predict_proba(df[features])

In [20]:
df['y_pred'] = y_pred
df['y_pred_proba'] = y_pred_proba[:, 1]

In [21]:
df.sort_values('y_pred_proba', ascending=False).head(10)

,vacancy_name,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,vacancy_description,resume_id,resume_title,resume_specialization,resume_last_position,...,resume_location,resume_gender,resume_applicant_status,resume_last_company_experience_months,location_matching,resume_skill_count_in_vacancy,last_position_in_vacancy,similarity_score_tfidf,y_pred,y_pred_proba
8423,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,54799,"Marketing, Sales, Project Management","['Менеджер по продажам, менеджер по работе с к...",EE F&HC Business (Marketing) Analyst,...,Москва,Мужчина,NDT,4,1,0,0.000000,0.000000,1,1.000000
2589,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,7092,"Эксперт ГИС, менеджер проекта",['Руководитель проектов'],Специалист географических информационных систем,...,Москва,Женщина,NDT,309,1,0,0.200000,0.104827,1,1.000000
20106,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,450698,директор,['Другое'],генеральный директор – директор по производству,...,Московская область,Мужчина,NDT,286,0,0,0.166667,0.021874,1,1.000000
5991,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,449402,Statistician / Modeling Manager,"['Sales manager, account manager']",analyst,...,Moscow Oblast,Женщина,NDT,0,0,0,0.000000,0.000000,1,1.000000
1315,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,8664598,DevOps/tech lead DevOps,['DevOps-инженер'],Архитектор автоматизации,...,Москва,Мужчина,Активно ищет работу,6,1,0,0.000000,0.013044,1,0.999952
1052,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,20903580,Аналитик данных (Data Analyst),"['BI-аналитик, аналитик данных', 'Продуктовый ...",Аналитик данных,...,Москва,Мужчина,Активно ищет работу,8,1,2,0.666667,0.336429,1,0.999914
1072,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,63289302,Аналитик,['Аналитик'],Аналитик,...,Москва,Мужчина,Рассматривает предложения,25,1,1,1.000000,0.295619,1,0.999813
1055,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,135124273,"Аналитик данных (SQL, Excel, Python)",['Аналитик'],Аналитик,...,Москва,Женщина,Активно ищет работу,13,1,0,1.000000,0.313912,1,0.999306
716,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,86893416,Аналитик данных,"['BI-аналитик, аналитик данных', 'Аналитик']",Ведущий специалист (группа отчетности),...,Новосибирск,Мужчина,Активно ищет работу,19,0,2,0.000000,0.256485,1,0.998675
406,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,Твои задачи:• Формирование баз данных для КЦ (...,54975055,"Программист, тестировщик, SEO-специалист, марк...","['BI-аналитик, аналитик данных', 'SMM-менеджер...",Руководитель отдела маркетинга и рекламы,...,Москва,Мужчина,Активно ищет работу,38,1,2,0.333333,0.284891,1,0.998551


In [24]:
df[['vacancy_description', 'resume_last_experience_description']]

,vacancy_description,resume_last_experience_description
0,Твои задачи:• Формирование баз данных для КЦ (...,Участие в проектах:\r\nСибинтек (SAP MDG (Mast...
1,Твои задачи:• Формирование баз данных для КЦ (...,"SAP S/4HANA (BTP, RAP, CDS, AMDP, ODATA V4, FI..."
2,Твои задачи:• Формирование баз данных для КЦ (...,ABAP-разработчик (SAP Solution Manager)\n•\tУч...
3,Твои задачи:• Формирование баз данных для КЦ (...,Внедрение SAP ISU в МРСК-Сибири.\r\nМиграция д...
4,Твои задачи:• Формирование баз данных для КЦ (...,Projects:\r\nParticipated in 9 projects in Rus...
...,...,...
20713,Твои задачи:• Формирование баз данных для КЦ (...,HSBC :Hong Kong and Shanghai Banking Corporati...
20714,Твои задачи:• Формирование баз данных для КЦ (...,Обслуживание и ремонт систем кондиционирования...
20715,Твои задачи:• Формирование баз данных для КЦ (...,Обучение вновь прибывших сотрудников.
20716,Твои задачи:• Формирование баз данных для КЦ (...,"Сопровождение VIP персон, организация встреч с..."


In [25]:
df['resume_last_experience_description'][0]

'Участие в проектах:\r\nСибинтек (SAP MDG (Master Data Governance),\r\nTELE2.\r\nРаботы выполняемые на проектах:\r\nРазработка функционала согласно спецификаций, модификация, расширение стандартной функциональности SAP (Enhancement, Customer-exit, Badi), написание отчетов (ALV), средства поиска. Формирование печатной документации (PDF, Excel), оптимизация программ, загрузка начальных данных (BAPI).'